<a href="https://colab.research.google.com/github/MostHumble/GeoFireTizi/blob/main/GeoFireTizi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import geopandas as gpd
import ee
from IPython.display import Image

In [ ]:
# You'll need to have access to the Earth Engine api
ee.Authenticate() 

In [7]:
ee.Initialize()

In [132]:
lat = 36.704642 # Tizi-Ouzou location
lon = 4.052215
# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2021-05-01'

# end date
end_date = '2022-07-01'

In [133]:
# getting the data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date) 

In [134]:
print('Total number:', landsat.size().getInfo()) 

Total number: 51


In [ ]:
landsat.first().getInfo()

In [136]:
# cloud coverage aka how "good" is the picture
landsat.first().get('CLOUD_COVER').getInfo() 

58.41

In [137]:
# date it was taken 
landsat.first().get('DATE_ACQUIRED').getInfo()  

'2021-05-10'

In [ ]:
# bands of the data from diffrent wavelenghts
landsat.first().bandNames().getInfo()

In [141]:
# put the images in a list
landsat_list = landsat.toList(landsat.size())

In [140]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [ ]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    # print the image info
    print('Image #',i,date,'Cloud cover:',cloud)
    
    # display the image
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

    # data to list
    this_data = [i,date,cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

In [143]:
threshold = df['Cloud Cover']<5

In [144]:
landsat_sequence =  list((df['Image #'][threshold].index))

In [145]:
print(len(landsat_sequence)) # number of pictures

14


In [149]:
roi = poi.buffer(20000) # meters

In [150]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [ ]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

In [ ]:
"""The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements,
 often from a space platform, assessing whether or not the target being observed contains live green vegetation."""

In [152]:
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [153]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 2 2021-06-11


Image # 6 2021-08-14


Image # 7 2021-08-30


Image # 12 2021-12-04


Image # 23 2022-05-29


...

Image # 38 2021-12-04


Image # 39 2021-12-20


Image # 41 2022-01-21


Image # 48 2022-05-13


Image # 49 2022-05-29


In [154]:
import folium

m = folium.Map(location=[lat,lon])

In [155]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [157]:
my_map.save('camp.html')